# Sistema de Recomendação de Arte - WikiArt
## Projeto Acadêmico: Inteligência Artificial

**Nome:** Gabriel Brito de França  
**Matrícula:** 211020867

---

## Roteiro do Projeto

1.  **Tema/Negócio:** Recomendação de obras de arte (pinturas, esculturas, gravuras)
2.  **20+ Produtos:** 10.000 obras de arte do dataset WikiArt
3.  **3 Características:** Artista, Estilo, Gênero  
4.  **Padrão de Avaliação:** Binário [0, 1] - Like/Dislike
5.  **Matriz de Utilidade:** 500+ linhas de interações usuário-obra
6.  **Modelo TF-IDF:** Score de relevância baseado em conteúdo
7.  **Interface:** Aplicativo Django + React para cadastro e recomendações

In [1]:
# Passo 1: Carregar Dataset WikiArt (10.000+ obras de arte)
from datasets import load_dataset

def load_wikiart_dataset():
    """
    Carrega o dataset WikiArt do HuggingFace
    Contém mais de 10.000 obras de arte com metadados
    """
    print("🎨 Carregando dataset WikiArt...")
    dataset = load_dataset("huggan/wikiart", streaming=True, split="train")
    print("✅ Dataset carregado com sucesso!")
    return dataset

# Carregar o dataset
dataset = load_wikiart_dataset()

# Verificar as colunas disponíveis (características)
print("📊 Colunas disponíveis no dataset:")
print(dataset.column_names)

🎨 Carregando dataset WikiArt...


Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

✅ Dataset carregado com sucesso!
📊 Colunas disponíveis no dataset:
['image', 'artist', 'genre', 'style']


## 1. Tema/Ramo de Negócio e Lista de Produtos

**Tema:** Sistema de Recomendação de Obras de Arte  
**Ramo:** Galeria de Arte Digital / E-commerce Cultural

### Características dos Produtos:
- **Característica 1:** Artista (Van Gogh, Picasso, Monet, etc.)
- **Característica 2:** Estilo Artístico (Impressionismo, Cubismo, Realismo, etc.)  
- **Característica 3:** Gênero (Retrato, Paisagem, Natureza Morta, etc.)

### Padrão de Avaliação:
- **Escala:** Binária [0, 1]
- **0:** Não curtiu a obra (Dislike)
- **1:** Curtiu a obra (Like)

### Dataset: 
Utilizaremos o dataset **WikiArt** disponível no hugging face

In [ ]:
# Passo 2: Organizar e Formatar os Dados (3 Características por Obra)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict

def process_streaming_dataset(dataset, max_items=10000):
    print(f"🔄 Processando dataset (máximo {max_items} obras)...")
    
    text_features = []  
    metadata = []      
    
    # Contadores para estatísticas
    genre_counts = defaultdict(int)
    artist_counts = defaultdict(int) 
    style_counts = defaultdict(int)
    
    for i, item in enumerate(dataset):
        if i >= max_items:
            break
            
        # Debug: mostrar primeiros 3 itens
        if i < 3:
            print(f"📋 Item {i}: {item}")
        
        # Extrair as 3 características obrigatórias
        features = []
        artist = None
        style = None
        genre = None

        # 1. ARTISTA (Característica 1)
        for field in ['artist', 'Artist', 'ARTIST']:
            if field in item and item[field]:
                artist = str(item[field]).strip()
                break
                
        # 2. ESTILO (Característica 2)  
        for field in ['style', 'Style', 'STYLE']:
            if field in item and item[field]:
                style = str(item[field]).strip()
                break
                
        # 3. GÊNERO (Característica 3)
        for field in ['genre', 'Genre', 'GENRE']:
            if field in item and item[field]:
                genre = str(item[field]).strip()
                break
        
        # Construir features para TF-IDF apenas com dados válidos
        if artist and artist.lower() not in ['none', 'null', '']:
            clean_artist = artist.replace(' ', '_').replace('-', '_')
            features.append(f"artist_{clean_artist}")
            artist_counts[artist] += 1
            
        if style and style.lower() not in ['none', 'null', '']:
            clean_style = style.replace(' ', '_').replace('-', '_')
            features.append(f"style_{clean_style}")
            style_counts[style] += 1
            
        if genre and genre.lower() not in ['none', 'null', '']:
            clean_genre = genre.replace(' ', '_').replace('-', '_')
            features.append(f"genre_{clean_genre}")
            genre_counts[genre] += 1
        
        # Criar feature de texto para TF-IDF
        if features:
            text_feature = " ".join(features)
        else:
            # Fallback para obras sem metadados
            text_feature = f"artwork_id_{i} category_general"
            
        text_features.append(text_feature)
        
        # Armazenar metadados estruturados (produto com 3 características)
        metadata.append({
            'id': i,
            'artist': artist if artist else 'Unknown',
            'style': style if style else 'Unknown',
            'genre': genre if genre else 'Unknown',
            'likes': 0 
        })
        
        # Indicador de progresso
        if (i + 1) % 1000 == 0:
            print(f"✅ Processadas {i + 1} obras...")
    
    # Estatísticas finais
    print(f"\n📊 RESUMO DO DATASET PROCESSADO:")
    print(f"✅ Total de obras processadas: {len(text_features)}")
    print(f"✅ Artistas únicos: {len(artist_counts)}")
    print(f"✅ Estilos únicos: {len(style_counts)}")
    print(f"✅ Gêneros únicos: {len(genre_counts)}")
    
    # Amostras das características extraídas
    print(f"\n🎨 AMOSTRAS DAS CARACTERÍSTICAS:")
    for i in range(min(5, len(text_features))):
        print(f"  Obra {i}: '{text_features[i]}'")
    
    return text_features, metadata, {
        'genres': dict(genre_counts),
        'artists': dict(artist_counts),
        'styles': dict(style_counts)
    }

In [ ]:
# Passo 3: Modelo de Recomendação baseado em Conteúdo usando TF-IDF

def train_streaming_model(max_items=10000):
    """
    Treina modelo de recomendação baseado em conteúdo usando TF-IDF
    Score TF-IDF indica características relevantes para recomendação
    """
    print("TREINANDO MODELO DE RECOMENDAÇÃO TF-IDF")
    
    # Carregar e processar dados
    dataset = load_wikiart_dataset()
    text_features, metadata, stats = process_streaming_dataset(dataset, max_items)
    
    # Validar se temos características válidas
    if not text_features or all(f.strip() == "" for f in text_features):
        raise ValueError("❌ Nenhuma característica válida encontrada no dataset")
    
    print("\n🔧 Configurando vetorizador TF-IDF...")
    
    # Configurar TF-IDF para score de relevância
    vectorizer = TfidfVectorizer(
        max_features=1000,    # Máximo 1000 características mais relevantes
        ngram_range=(1, 1), 
        min_df=1,            
        max_df=0.99,  
        lowercase=True,       # Normalizar para minúsculas
        token_pattern=r'\b\w+\b'  # Padrão de tokenização
    )
    
    try:
        # Treinar TF-IDF e gerar matriz de características
        print("📈 Calculando scores TF-IDF...")
        tfidf_matrix = vectorizer.fit_transform(text_features)
        
        print(f"✅ Matriz TF-IDF criada: {tfidf_matrix.shape}")
        print(f"✅ Vocabulário (características): {len(vectorizer.vocabulary_)} termos")
        
        # Testar o sistema de recomendação
        test_recommendations_streaming(tfidf_matrix, metadata, test_id=0)
        
        return vectorizer, tfidf_matrix, metadata, stats
        
    except ValueError as e:
        print(f"❌ Falha no treinamento TF-IDF: {e}")
        print("🔄 Criando matriz de similaridade alternativa...")
        
        # Fallback: matriz identidade para testes
        n_samples = len(text_features)
        fallback_matrix = np.eye(n_samples)
        
        return None, fallback_matrix, metadata, stats

In [4]:
# Passo 4: Sistema de Recomendação e Testes

class StreamingRecommender:
    """
    Sistema de recomendação baseado em conteúdo usando TF-IDF
    Considera perfil do usuário (obras curtidas) para personalizar recomendações
    """
    
    def __init__(self, vectorizer=None, tfidf_matrix=None, metadata=None):
        self.vectorizer = vectorizer
        self.tfidf_matrix = tfidf_matrix
        self.metadata = metadata
    
    def get_recommendations(self, artwork_id, user_likes=None, n_recommendations=10):
        """
        Gera recomendações baseadas em:
        1. Similaridade TF-IDF entre obras
        2. Perfil do usuário (obras curtidas anteriormente)
        """
        if self.tfidf_matrix is None:
            return []
        
        # Calcular similaridade base usando cosine similarity
        similarity_scores = cosine_similarity(
            self.tfidf_matrix[artwork_id:artwork_id+1], 
            self.tfidf_matrix
        ).flatten()
        
        # Personalizar com base no perfil do usuário (obras curtidas)
        if user_likes:
            print(f"🔍 Personalizando com base em {len(user_likes)} obras curtidas...")
            for liked_id in user_likes:
                if liked_id < len(similarity_scores):
                    # Calcular similaridade com obra curtida
                    liked_similarity = cosine_similarity(
                        self.tfidf_matrix[liked_id:liked_id+1],
                        self.tfidf_matrix
                    ).flatten()
                    # Boost baseado em obras similares às curtidas
                    similarity_scores += 0.3 * liked_similarity
        
        # Ordenar por relevância (score TF-IDF)
        similar_indices = similarity_scores.argsort()[::-1]
        recommendations = []
        
        # Selecionar top recomendações
        for idx in similar_indices:
            if idx != artwork_id and len(recommendations) < n_recommendations:
                artwork_info = self.metadata[idx].copy()
                artwork_info['similarity_score'] = float(similarity_scores[idx])
                recommendations.append(artwork_info)
        
        return recommendations

def test_recommendations_streaming(tfidf_matrix, metadata, test_id=0, n_recs=5):
    """Testar sistema de recomendação com obra específica"""
    print(f"\n🧪 TESTANDO RECOMENDAÇÕES PARA OBRA {test_id}:")
    print(f"🎨 Obra teste: {metadata[test_id]}")
    
    # Calcular similaridades
    similarities = cosine_similarity(
        tfidf_matrix[test_id:test_id+1], 
        tfidf_matrix
    ).flatten()
    
    # Top recomendações
    similar_indices = similarities.argsort()[::-1][1:n_recs+1]
    
    print(f"\n🏆 TOP {n_recs} RECOMENDAÇÕES:")
    for i, idx in enumerate(similar_indices):
        artwork = metadata[idx]
        print(f"{i+1}. Artista: {artwork['artist']}, Estilo: {artwork['style']}, "
              f"Gênero: {artwork['genre']} (Score: {similarities[idx]:.3f})")
    
    return similar_indices

# EXECUTAR TREINAMENTO E TESTES
print("🚀 INICIANDO TREINAMENTO DO MODELO...")
vectorizer, tfidf_matrix, metadata, stats = train_streaming_model(max_items=10000)

# Criar sistema de recomendação
recommender = StreamingRecommender(vectorizer, tfidf_matrix, metadata)

# Testar com diferentes obras
print("\n" + "="*80)
print("🎯 TESTES DO SISTEMA DE RECOMENDAÇÃO")
print("="*80)

for test_id in [0, 10, 100]:
    recommendations = recommender.get_recommendations(test_id, n_recommendations=3)
    print(f"\n🎨 Recomendações para obra {test_id}:")
    for i, rec in enumerate(recommendations):
        print(f"  {i+1}. {rec['artist']} - {rec['style']} (Score: {rec['similarity_score']:.3f})")

🚀 INICIANDO TREINAMENTO DO MODELO...
🤖 TREINANDO MODELO DE RECOMENDAÇÃO TF-IDF
🎨 Carregando dataset WikiArt...


Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

✅ Dataset carregado com sucesso!
🔄 Processando dataset (máximo 10000 obras)...
📋 Item 0: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=750x597 at 0x1C7D1CE13A0>, 'artist': 22, 'genre': 4, 'style': 21}
📋 Item 1: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1382x1659 at 0x1C7D1D70920>, 'artist': 20, 'genre': 7, 'style': 4}
📋 Item 2: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1382x1673 at 0x1C7D1D70A40>, 'artist': 16, 'genre': 6, 'style': 20}
📋 Item 0: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=750x597 at 0x1C7D1CE13A0>, 'artist': 22, 'genre': 4, 'style': 21}
📋 Item 1: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1382x1659 at 0x1C7D1D70920>, 'artist': 20, 'genre': 7, 'style': 4}
📋 Item 2: {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1382x1673 at 0x1C7D1D70A40>, 'artist': 16, 'genre': 6, 'style': 20}
✅ Processadas 1000 obras...
✅ Processadas 1000 obras...
✅ Processadas

## 2. Matriz de Utilidade (500+ linhas conforme requisito)

A matriz de utilidade contém as avaliações dos usuários sobre as obras de arte.
**Formato:** user_id, artwork_id, rating [0,1]  
**Requisito:** Mínimo 500 linhas de interações

In [5]:
# Passo 2: Gerar Matriz de Utilidade (500+ linhas conforme requisito acadêmico)

import pandas as pd
import numpy as np
import random
from collections import defaultdict

def generate_utility_matrix(metadata, min_interactions=500, n_users=100):
    """
    GERAR MATRIZ DE UTILIDADE REAL - REQUISITO ACADÊMICO
    
    Parâmetros:
    - metadata: lista de obras de arte com 3 características
    - min_interactions: mínimo 500 linhas (requisito)
    - n_users: número de usuários simulados
    
    Retorna:
    - DataFrame com user_id, artwork_id, rating [0,1]
    """
    print("🎯 GERANDO MATRIZ DE UTILIDADE (REQUISITO: 500+ LINHAS)")
    print(f"📊 Target: {min_interactions}+ interações com {n_users} usuários")
    print(f"🎨 Obras disponíveis: {len(metadata)}")
    
    # Criar perfis de usuário com preferências realistas
    user_profiles = []
    for user_id in range(n_users):
        # Cada usuário tem preferências por artistas, estilos e gêneros
        all_artists = list(set(m['artist'] for m in metadata if m['artist'] != 'Unknown'))
        all_styles = list(set(m['style'] for m in metadata if m['style'] != 'Unknown'))
        all_genres = list(set(m['genre'] for m in metadata if m['genre'] != 'Unknown'))
        
        preferred_artists = random.sample(all_artists, k=min(random.randint(2, 5), len(all_artists)))
        preferred_styles = random.sample(all_styles, k=min(random.randint(1, 3), len(all_styles)))
        preferred_genres = random.sample(all_genres, k=min(random.randint(1, 3), len(all_genres)))
        
        user_profiles.append({
            'user_id': user_id,
            'preferred_artists': preferred_artists,
            'preferred_styles': preferred_styles,
            'preferred_genres': preferred_genres,
            'exploration_rate': random.uniform(0.1, 0.4)  # Taxa de exploração
        })
    
    # Gerar interações usuário-obra
    interactions = []
    interaction_count = 0
    
    print("🔄 Gerando interações baseadas em preferências...")
    
    while interaction_count < min_interactions:
        for user_id in range(n_users):
            if interaction_count >= min_interactions:
                break
            
            user_profile = user_profiles[user_id]
            
            # Cada usuário interage com 8-15 obras
            n_interactions_user = random.randint(8, 15)
            
            # Selecionar obras aleatórias para este usuário
            sampled_artworks = random.sample(metadata, k=min(n_interactions_user, len(metadata)))
            
            for artwork in sampled_artworks:
                if interaction_count >= min_interactions:
                    break
                
                # Calcular probabilidade de like baseada nas 3 características
                like_probability = 0.1  # Probabilidade base
                
                # BOOST POR CARACTERÍSTICA 1: ARTISTA
                if artwork['artist'] in user_profile['preferred_artists']:
                    like_probability += 0.6
                
                # BOOST POR CARACTERÍSTICA 2: ESTILO
                if artwork['style'] in user_profile['preferred_styles']:
                    like_probability += 0.4
                    
                # BOOST POR CARACTERÍSTICA 3: GÊNERO
                if artwork['genre'] in user_profile['preferred_genres']:
                    like_probability += 0.4
                
                # Fator de exploração
                like_probability += user_profile['exploration_rate']
                
                # Garantir probabilidade entre 0 e 1
                like_probability = min(1.0, like_probability)
                
                # Gerar rating binário [0,1] conforme padrão definido
                rating = 1 if random.random() < like_probability else 0
                
                # Adicionar interação à matriz
                interactions.append({
                    'user_id': user_id,
                    'artwork_id': artwork['id'],
                    'rating': rating,  # Padrão [0,1]: 0=dislike, 1=like
                    'artist': artwork['artist'],
                    'style': artwork['style'],
                    'genre': artwork['genre']
                })
                
                interaction_count += 1
    
    # Converter para DataFrame (formato da matriz de utilidade)
    utility_df = pd.DataFrame(interactions)
    
    # Estatísticas da matriz gerada
    total_interactions = len(utility_df)
    unique_users = utility_df['user_id'].nunique()
    unique_artworks = utility_df['artwork_id'].nunique()
    like_rate = utility_df['rating'].mean()
    
    print(f"\n📊 ESTATÍSTICAS DA MATRIZ DE UTILIDADE:")
    print(f"✅ Total de interações: {total_interactions}")
    print(f"✅ Usuários únicos: {unique_users}")
    print(f"✅ Obras únicas: {unique_artworks}")
    print(f"✅ Taxa de likes: {like_rate:.2%}")
    print(f"✅ Interações por usuário: {total_interactions/unique_users:.1f}")
    print(f"✅ REQUISITO ACADÊMICO: {'✅ ATENDIDO' if total_interactions >= 500 else '❌ NÃO ATENDIDO'}")
    
    return utility_df, user_profiles

# Gerar a matriz se já temos os metadados
if 'metadata' in locals() and len(metadata) > 0:
    print("🚀 Gerando matriz de utilidade...")
    utility_matrix, user_profiles = generate_utility_matrix(metadata, min_interactions=600)
    
    print(f"\n📋 AMOSTRA DA MATRIZ DE UTILIDADE:")
    print(utility_matrix.head(10))
    
    print(f"\n💾 DISTRIBUIÇÃO DE RATINGS:")
    print(utility_matrix['rating'].value_counts().sort_index())
    
else:
    print("❌ Execute primeiro as células de carregamento dos dados")

🚀 Gerando matriz de utilidade...
🎯 GERANDO MATRIZ DE UTILIDADE (REQUISITO: 500+ LINHAS)
📊 Target: 600+ interações com 100 usuários
🎨 Obras disponíveis: 10000
🔄 Gerando interações baseadas em preferências...

📊 ESTATÍSTICAS DA MATRIZ DE UTILIDADE:
✅ Total de interações: 600
✅ Usuários únicos: 54
✅ Obras únicas: 580
✅ Taxa de likes: 54.33%
✅ Interações por usuário: 11.1
✅ REQUISITO ACADÊMICO: ✅ ATENDIDO

📋 AMOSTRA DA MATRIZ DE UTILIDADE:
   user_id  artwork_id  rating artist style genre
0        0        4592       0     18     9     2
1        0        4291       0      1    21     6
2        0        8411       1      7    23     7
3        0        7042       0     11    21     6
4        0        1121       0      9    23    10
5        0        6438       1     15     7     6
6        0        5171       0      4    12     4
7        0        5599       1     14    24     4
8        0        9908       0     20     4     6
9        0        4913       0      5    12     6

💾 DISTRIB

In [6]:
# Salvar Matriz de Utilidade como CSV (conforme requisito de disponibilização)

def save_utility_matrix_files(utility_df, models_dir="../../models"):
    """
    Salvar matriz de utilidade e estatísticas em arquivos CSV
    REQUISITO: "deve ser guardada e disponibilizada para efeito do projeto"
    """
    import os
    import json
    
    os.makedirs(models_dir, exist_ok=True)
    
    print(f"💾 SALVANDO MATRIZ DE UTILIDADE EM {models_dir}")
    
    # 1. ARQUIVO PRINCIPAL: Matriz de Utilidade (500+ linhas)
    utility_file = f"{models_dir}/utility_matrix.csv"
    utility_df.to_csv(utility_file, index=False)
    print(f"✅ Matriz de utilidade salva: {utility_file} ({len(utility_df)} linhas)")
    
    # 2. FORMATO PIVOT: Matriz usuário x obra
    pivot_df = utility_df.pivot_table(
        index='user_id', 
        columns='artwork_id', 
        values='rating', 
        fill_value=0
    )
    pivot_file = f"{models_dir}/utility_matrix_pivot.csv"
    pivot_df.to_csv(pivot_file)
    print(f"✅ Matriz pivot salva: {pivot_file} ({pivot_df.shape[0]}x{pivot_df.shape[1]})")
    
    # 3. ESTATÍSTICAS POR USUÁRIO
    user_stats = utility_df.groupby('user_id').agg({
        'rating': ['count', 'sum', 'mean'],
        'artwork_id': 'nunique'
    }).round(3)
    user_stats.columns = ['total_interactions', 'total_likes', 'like_rate', 'unique_artworks']
    user_stats.to_csv(f"{models_dir}/user_statistics.csv")
    print(f"✅ Estatísticas de usuários salvas: {len(user_stats)} usuários")
    
    # 4. ESTATÍSTICAS POR OBRA
    artwork_stats = utility_df.groupby('artwork_id').agg({
        'rating': ['count', 'sum', 'mean'],
        'user_id': 'nunique',
        'artist': 'first',
        'style': 'first',
        'genre': 'first'
    }).round(3)
    artwork_stats.columns = ['total_views', 'total_likes', 'popularity', 'unique_users', 'artist', 'style', 'genre']
    artwork_stats.to_csv(f"{models_dir}/artwork_statistics.csv")
    print(f"✅ Estatísticas de obras salvas: {len(artwork_stats)} obras")
    
    # 5. RESUMO EXECUTIVO
    summary = {
        "project_info": {
            "description": "Sistema de Recomendação de Arte - Projeto Acadêmico",
            "student": "Gabriel Brito de França - 211020867",
            "dataset": "WikiArt (10.000+ obras)",
            "characteristics": ["artist", "style", "genre"],
            "rating_pattern": "[0,1] - Binary Like/Dislike"
        },
        "utility_matrix_stats": {
            "total_interactions": len(utility_df),
            "unique_users": utility_df['user_id'].nunique(),
            "unique_artworks": utility_df['artwork_id'].nunique(),
            "like_rate": float(utility_df['rating'].mean()),
            "academic_requirement": "500+ lines",
            "requirement_met": len(utility_df) >= 500
        },
        "files_generated": {
            "utility_matrix.csv": f"{len(utility_df)} interactions",
            "utility_matrix_pivot.csv": f"{pivot_df.shape[0]}x{pivot_df.shape[1]} matrix",
            "user_statistics.csv": f"{len(user_stats)} users",
            "artwork_statistics.csv": f"{len(artwork_stats)} artworks"
        }
    }
    
    with open(f"{models_dir}/project_summary.json", 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"✅ Resumo do projeto salvo")
    
    # Mostrar amostras dos arquivos salvos
    print(f"\n📋 VERIFICAÇÃO DOS ARQUIVOS SALVOS:")
    print(f"\n💾 utility_matrix.csv (primeiras 5 linhas):")
    print(utility_df.head())
    
    print(f"\n📊 Resumo estatístico:")
    print(f"- Total de interações: {len(utility_df)}")
    print(f"- Requisito acadêmico (500+ linhas): {'✅ ATENDIDO' if len(utility_df) >= 500 else '❌ NÃO ATENDIDO'}")
    
    return summary

# Salvar a matriz se ela foi gerada
if 'utility_matrix' in locals():
    summary = save_utility_matrix_files(utility_matrix)
    print(f"\n🎯 MATRIZ DE UTILIDADE DISPONIBILIZADA CONFORME REQUISITO!")
else:
    print("❌ Gere a matriz de utilidade primeiro executando a célula anterior")

💾 SALVANDO MATRIZ DE UTILIDADE EM ../../models
✅ Matriz de utilidade salva: ../../models/utility_matrix.csv (600 linhas)
✅ Matriz pivot salva: ../../models/utility_matrix_pivot.csv (54x580)
✅ Estatísticas de usuários salvas: 54 usuários
✅ Estatísticas de obras salvas: 580 obras
✅ Resumo do projeto salvo

📋 VERIFICAÇÃO DOS ARQUIVOS SALVOS:

💾 utility_matrix.csv (primeiras 5 linhas):
   user_id  artwork_id  rating artist style genre
0        0        4592       0     18     9     2
1        0        4291       0      1    21     6
2        0        8411       1      7    23     7
3        0        7042       0     11    21     6
4        0        1121       0      9    23    10

📊 Resumo estatístico:
- Total de interações: 600
- Requisito acadêmico (500+ linhas): ✅ ATENDIDO

🎯 MATRIZ DE UTILIDADE DISPONIBILIZADA CONFORME REQUISITO!
✅ Matriz pivot salva: ../../models/utility_matrix_pivot.csv (54x580)
✅ Estatísticas de usuários salvas: 54 usuários
✅ Estatísticas de obras salvas: 580 obras


In [7]:
# Passo 5: Salvar Modelo Treinado para Integração com Interface

import os
import pickle
import json
from scipy.sparse import save_npz

def save_model_for_application():
    """
    Salvar componentes do modelo treinado para uso na interface/aplicativo
    """
    # Criar diretório de modelos
    models_dir = "../../models"
    os.makedirs(models_dir, exist_ok=True)
    
    print("💾 SALVANDO MODELO PARA INTERFACE/APLICATIVO...")
    
    # Salvar vetorizador TF-IDF
    with open(f"{models_dir}/vectorizer.pkl", 'wb') as f:
        pickle.dump(vectorizer, f)
    print("✅ Vetorizador TF-IDF salvo")
    
    # Salvar matriz TF-IDF (formato esparso para eficiência)
    save_npz(f"{models_dir}/tfidf_matrix.npz", tfidf_matrix)
    print("✅ Matriz TF-IDF salva")
    
    # Salvar metadados das obras (3 características + ID)
    with open(f"{models_dir}/metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    print("✅ Metadados das obras salvos")
    
    # Informações do modelo
    model_info = {
        "project_title": "Sistema de Recomendação de Arte",
        "student": "Gabriel Brito de França - 211020867",
        "n_artworks": len(metadata),
        "n_features": tfidf_matrix.shape[1],
        "vocabulary_size": len(vectorizer.vocabulary_),
        "unique_artists": len(set(m['artist'] for m in metadata)),
        "unique_genres": len(set(m['genre'] for m in metadata)),
        "unique_styles": len(set(m['style'] for m in metadata)),
        "model_version": "1.0",
        "characteristics": {
            "1": "artist",
            "2": "style", 
            "3": "genre"
        },
        "rating_pattern": "[0,1] - Binary Like/Dislike",
        "algorithm": "TF-IDF + Cosine Similarity"
    }
    
    with open(f"{models_dir}/model_info.json", 'w') as f:
        json.dump(model_info, f, indent=2)
    print("✅ Informações do modelo salvas")
    
    print(f"\n🎯 Modelo salvo em: {models_dir}")
    print(f"📊 {model_info['n_artworks']} obras, {model_info['vocabulary_size']} características")
    
    return models_dir

# Executar salvamento se o modelo foi treinado
if 'vectorizer' in locals() and vectorizer is not None:
    save_model_for_application()
    print("\n✅ MODELO PRONTO PARA INTEGRAÇÃO COM INTERFACE!")
else:
    print("❌ Treine o modelo primeiro executando as células anteriores")

💾 SALVANDO MODELO PARA INTERFACE/APLICATIVO...
✅ Vetorizador TF-IDF salvo
✅ Matriz TF-IDF salva
✅ Metadados das obras salvos
✅ Informações do modelo salvas

🎯 Modelo salvo em: ../../models
📊 10000 obras, 48 características

✅ MODELO PRONTO PARA INTEGRAÇÃO COM INTERFACE!
✅ Metadados das obras salvos
✅ Informações do modelo salvas

🎯 Modelo salvo em: ../../models
📊 10000 obras, 48 características

✅ MODELO PRONTO PARA INTEGRAÇÃO COM INTERFACE!


In [8]:
# Passo 6: Testar Integração com Interface/Aplicativo

def test_application_integration():
    """
    Testar que o modelo pode ser carregado corretamente pela interface
    Simula como o aplicativo Django + React carregará os dados
    """
    # Simular carregamento como a interface faria
    import json
    from scipy.sparse import load_npz
    import pickle
    
    models_dir = "../../models"
    
    print("🧪 TESTANDO INTEGRAÇÃO COM INTERFACE/APLICATIVO")
    
    # Carregar como a interface faria
    with open(f"{models_dir}/vectorizer.pkl", 'rb') as f:
        test_vectorizer = pickle.load(f)
    
    test_matrix = load_npz(f"{models_dir}/tfidf_matrix.npz")
    
    with open(f"{models_dir}/metadata.json", 'r') as f:
        test_metadata = json.load(f)
    
    print(f"✅ Vetorizador carregado: {len(test_vectorizer.vocabulary_)} características")
    print(f"✅ Matriz carregada: {test_matrix.shape}")
    print(f"✅ Metadados carregados: {len(test_metadata)} obras")
    
    # Testar recomendação (simula um usuário novo na interface)
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = cosine_similarity(test_matrix[0:1], test_matrix).flatten()
    top_3 = similarities.argsort()[::-1][1:4]
    
    print(f"\n🎯 Teste de recomendação para usuário novo (obra 0):")
    for i, idx in enumerate(top_3):
        artwork = test_metadata[idx]
        print(f"   {i+1}. {artwork['artist']} | {artwork['genre']} | {artwork['style']}")
    
    print("\n✅ INTEGRAÇÃO COM INTERFACE FUNCIONANDO!")
    return True

# Executar teste
test_application_integration()

🧪 TESTANDO INTEGRAÇÃO COM INTERFACE/APLICATIVO
✅ Vetorizador carregado: 48 características
✅ Matriz carregada: (10000, 48)
✅ Metadados carregados: 10000 obras

🎯 Teste de recomendação para usuário novo (obra 0):
   1. 22 | 4 | 21
   2. 22 | 4 | 21
   3. 22 | 4 | 21

✅ INTEGRAÇÃO COM INTERFACE FUNCIONANDO!
✅ Vetorizador carregado: 48 características
✅ Matriz carregada: (10000, 48)
✅ Metadados carregados: 10000 obras

🎯 Teste de recomendação para usuário novo (obra 0):
   1. 22 | 4 | 21
   2. 22 | 4 | 21
   3. 22 | 4 | 21

✅ INTEGRAÇÃO COM INTERFACE FUNCIONANDO!


True

## 6. Interface e Aplicativo para Novos Usuários

### 📱 Sistema Implementado: Django + React

**Backend (Django):**
- API REST para autenticação de usuários
- Sistema de recomendação integrado com TF-IDF
- Gerenciamento de perfis e avaliações
- Endpoints para registro e login

**Frontend (React):**
- Interface responsiva para cadastro de novos usuários
- Galeria de obras de arte interativa
- Sistema de likes/dislikes (padrão binário [0,1])
- Recebimento de recomendações personalizadas

### 🔄 Fluxo do Usuário:
1. **Cadastro:** Novo usuário se registra na plataforma
2. **Perfil:** Usuário avalia algumas obras iniciais (sistema binário)
3. **Recomendações:** Sistema TF-IDF gera recomendações baseadas no perfil
4. **Refinamento:** Usuário continua avaliando, melhorando as recomendações

### 🎯 Integração Completa:
- Modelo TF-IDF treinado neste notebook ✅
- Matriz de utilidade 500+ linhas ✅  
- Interface web funcional ✅
- Sistema de cadastro e perfil ✅

## ✅ Resumo de Conformidade com o Roteiro Acadêmico

### 1. ✅ Tema/Ramo de Negócio e Lista de Produtos:
- **Tema:** Sistema de Recomendação de Obras de Arte
- **Produtos:** 10.000+ obras de arte (muito superior aos 20 mínimos)
- **Características:** Artista, Estilo, Gênero (3 características obrigatórias)
- **Padrão:** Binário [0, 1] - Like/Dislike

### 2. ✅ Organização e Formatação dos Dados:
- Dados organizados em metadata estruturada
- Matriz de utilidade com 500+ linhas gerada e salva
- Critérios realistas baseados em preferências por características
- Dados disponibilizados em CSV conforme requisito

### 3. ✅ Modelo TF-IDF de Recomendação:
- Algoritmo TF-IDF implementado para score de relevância
- Características transformadas em features vetorizadas
- Sistema de recomendação baseado em conteúdo funcional
- Personalização baseada no perfil do usuário

### 6. ✅ Interface e Aplicativo:
- Interface web Django + React implementada
- Sistema de cadastro de novos usuários
- Preenchimento de perfil através de avaliações
- Recebimento de recomendações personalizadas

### 📝 Documentação:
- ✅ Notebook Jupyter com comentários em todas as etapas
- ✅ Instruções claras para execução
- ✅ Nome e matrícula identificados: Gabriel Brito de França - 211020867
- ✅ Código pronto para entrega acadêmica